# Pipeline de Pré-processamento - Tic Tac Toe
**Author**: Gabriela Dellamora Paim

**Version**: 04/04/2025

**Python Ver**: 3.10.11

In [ ]:
import pandas as pd
import random

PATH_OLD = './data_old/tic-tac-toe.data'
PATH_NEW = './data.csv'
X_WIN   = X = '1'
O_WIN   = O ='-1'
DRAW    = '0'
ONGOING = '~'

# Tratar Dados Existentes

In [34]:
df_raw = pd.read_csv(PATH_OLD, index_col=False)
df_raw.columns = ['1', '2', '3', '4', '5', '6', '7', '8', '9', 'category']
df_raw.describe

<bound method NDFrame.describe of      1  2  3  4  5  6  7  8  9  category
0    x  x  x  x  o  o  o  x  o  positive
1    x  x  x  x  o  o  o  o  x  positive
2    x  x  x  x  o  o  o  b  b  positive
3    x  x  x  x  o  o  b  o  b  positive
4    x  x  x  x  o  o  b  b  o  positive
..  .. .. .. .. .. .. .. .. ..       ...
952  o  x  x  x  o  o  o  x  x  negative
953  o  x  o  x  x  o  x  o  x  negative
954  o  x  o  x  o  x  x  o  x  negative
955  o  x  o  o  x  x  x  o  x  negative
956  o  o  x  x  x  o  o  x  x  negative

[957 rows x 10 columns]>

In [31]:
'''
Como o dataset apresenta todas opções possíveis de tabuleiro finalizado em que o X começa, vamos inverter os valores
e ter todas opções possíveis de tabuleiro finalizado em que Y começa
'''
# Como temos empates e derrotas nesse cara, vamos tratar separado, depois que conseguirmos remover todas observações de df_duplicate do df_neg
df_neg = df_raw[df_raw.category == 'negative']
df_neg.describe

<bound method NDFrame.describe of      1  2  3  4  5  6  7  8  9  category
625  x  x  o  x  x  o  o  b  o  negative
626  x  x  o  x  x  o  b  o  o  negative
627  x  x  o  x  x  b  o  o  o  negative
628  x  x  o  x  o  x  o  o  b  negative
629  x  x  o  x  o  x  o  b  o  negative
..  .. .. .. .. .. .. .. .. ..       ...
952  o  x  x  x  o  o  o  x  x  negative
953  o  x  o  x  x  o  x  o  x  negative
954  o  x  o  x  o  x  x  o  x  negative
955  o  x  o  o  x  x  x  o  x  negative
956  o  o  x  x  x  o  o  x  x  negative

[332 rows x 10 columns]>

In [37]:

# Vamos remover 'negatives' pra não ter com o que se preocupar por enquanto. Depois disso, vamos inverter as observações e salvar no df_duplicate
df = df_raw[df_raw.category != 'negative']
df.replace(to_replace={'positive' : X_WIN})
df.describe

<bound method NDFrame.describe of      1  2  3  4  5  6  7  8  9  category
0    x  x  x  x  o  o  o  x  o  positive
1    x  x  x  x  o  o  o  o  x  positive
2    x  x  x  x  o  o  o  b  b  positive
3    x  x  x  x  o  o  b  o  b  positive
4    x  x  x  x  o  o  b  b  o  positive
..  .. .. .. .. .. .. .. .. ..       ...
620  b  b  b  x  x  x  o  b  o  positive
621  b  b  b  x  x  x  b  o  o  positive
622  b  b  b  o  o  b  x  x  x  positive
623  b  b  b  o  b  o  x  x  x  positive
624  b  b  b  b  o  o  x  x  x  positive

[625 rows x 10 columns]>

In [39]:

df_duplicate = df.replace(to_replace={'o':'x', 'x':'o', X_WIN:O_WIN})
print(F'BEFORE={df.shape}')
df = pd.concat([df, df_duplicate], ignore_index=True)
print(F'AFTER={df.shape}')

BEFORE=(1250, 10)
AFTER=(2500, 10)


In [ ]:
# Precisamos tratar os valores em df_neg


In [13]:
df['category'].replace(to_replace={'positive' : X_WIN, 'negative' : O_WIN, 'draw' : DRAW, 'ongoing' : ONGOING}, inplace=True) # FIXME corrigir O_WIN e DRAW (negative)
df.replace(to_replace={'o' : O, 'x' : X, 'b' : 0}, inplace=True)
df

,1,2,3,4,5,6,7,8,9,category
0,1,1,1,1,-1,-1,-1,1,-1,1
1,1,1,1,1,-1,-1,-1,-1,1,1
2,1,1,1,1,-1,-1,-1,0,0,1
3,1,1,1,1,-1,-1,0,-1,0,1
4,1,1,1,1,-1,-1,0,0,-1,1
...,...,...,...,...,...,...,...,...,...,...
952,-1,1,1,1,-1,-1,-1,1,1,-1
953,-1,1,-1,1,1,-1,1,-1,1,-1
954,-1,1,-1,1,-1,1,1,-1,1,-1
955,-1,1,-1,-1,1,1,1,-1,1,-1


# Gerar Novos Dados

In [63]:
class Board:
    def __init__(self):
        self.board      = [[0,0,0], [0,0,0], [0,0,0]]
        self.moves_left = 9
        self.status     = ONGOING

    def move(self, symbol : int):
        if self.moves_left <= 0:
            self.update_status()
            return
        x, y = self.gen_indexes()
        self.board[x][y] = symbol
        self.moves_left -= 1
        self.update_status()

    def update_status(self):
        if self.moves_left <= 0:
            self.status = DRAW
        else:
            self.status = self.check_wins()

    def check_wins(self):
        # Diagonais
        if self.board[0][0] == self.board[1][1] == self.board[2][2] != 0:
            return self.board[1][1]
        if self.board[0][2] == self.board[1][1] == self.board[2][0] != 0:
            return self.board[1][1]
        # Linhas e colunas
        for i in range(3):
            if self.board[0][i] == self.board[1][i] == self.board[2][i] != 0:
                return self.board[0][i]
            if self.board[i][0] == self.board[i][1] == self.board[i][2] != 0:
                return self.board[i][0]
        return ONGOING

    def gen_indexes(self) -> tuple:
        while True:
            x, y = random.randint(0, 2), random.randint(0, 2)
            if self.board[x][y] == 0:
                return x, y

    def observation(self):
        flat = [cell for row in self.board for cell in row]
        flat.append(self.status)
        return flat

In [ ]:
def generate_new_movements(df: pd.DataFrame, repeat=1000):
    ran_var = 5
    new_data = []

    for _ in range(repeat//2):
        board = Board()

        while True:
            board.move(-1)
            if board.status != ONGOING:
                break
            new_data.append(board.observation()) if random.randrange(ran_var) == 0 else None

            board.move(1)
            if board.status != ONGOING:
                break
            new_data.append(board.observation()) if random.randrange(ran_var) == 0 else None

        new_data.append(board.observation())

    for _ in range(repeat//2):
        board = Board()

        while True:
            board.move(1)
            if board.status != ONGOING:
                break
            new_data.append(board.observation()) if random.randrange(ran_var) == 0 else None

            board.move(-1)
            if board.status != ONGOING:
                break
            new_data.append(board.observation()) if random.randrange(ran_var) == 0 else None

        new_data.append(board.observation())

    new_df = pd.DataFrame(new_data, columns=df.columns)
    return pd.concat([df, new_df], ignore_index=True)

'''
Tenho medo de que se observarmos todas ações até um resultado, isso poderá gerar um viés.
Por isso, adicionei uma "aleatoriedade" pra amostragem de tabuleiros "ONGOING"
'''

In [71]:
df_expanded = generate_new_movements(df, 5000)
print(f'Antes={df.size} : Depois={df_expanded.size}')

Antes=9570 : Depois=85380


# Balancear os dados
## Garantindo que existe amostra proporcional de cada (25% de cada categoria)

# Exportar Dados

In [73]:
df_expanded.to_csv(PATH_NEW, index=False)
df_expanded.sample(50)

,1,2,3,4,5,6,7,8,9,category
2208,0,0,0,0,0,1,0,0,-1,~
3334,1,1,-1,-1,1,1,-1,-1,-1,0
3342,-1,0,0,0,-1,1,0,-1,1,~
8160,-1,1,-1,-1,1,1,1,0,-1,~
4841,0,0,0,-1,0,0,0,0,0,~
7510,-1,0,-1,1,1,0,0,0,-1,~
1307,0,0,0,0,0,-1,1,1,-1,~
4576,0,0,1,0,0,0,0,0,-1,~
7218,1,0,-1,0,0,0,0,0,-1,~
5618,1,0,0,-1,0,0,0,1,-1,~
